<img align="left" src="https://lever-client-logos.s3.amazonaws.com/864372b1-534c-480e-acd5-9711f850815c-1524247202159.png" width=200>
<br></br>
<br></br>

## *Data Science Unit 4 Sprint 3 Assignment 1*

# Recurrent Neural Networks and Long Short Term Memory (LSTM)

![Monkey at a typewriter](https://upload.wikimedia.org/wikipedia/commons/thumb/3/3c/Chimpanzee_seated_at_typewriter.jpg/603px-Chimpanzee_seated_at_typewriter.jpg)

It is said that [infinite monkeys typing for an infinite amount of time](https://en.wikipedia.org/wiki/Infinite_monkey_theorem) will eventually type, among other things, the complete works of Wiliam Shakespeare. Let's see if we can get there a bit faster, with the power of Recurrent Neural Networks and LSTM.

This text file contains the complete works of Shakespeare: https://www.gutenberg.org/files/100/100-0.txt

Use it as training data for an RNN - you can keep it simple and train character level, and that is suggested as an initial approach.

Then, use that trained RNN to generate Shakespearean-ish text. Your goal - a function that can take, as an argument, the size of text (e.g. number of characters or lines) to generate, and returns generated text of that size.

Note - Shakespeare wrote an awful lot. It's OK, especially initially, to sample/use smaller data and parameters, so you can have a tighter feedback loop when you're trying to get things running. Then, once you've got a proof of concept - start pushing it more!

In [2]:
# TODO - Words, words, mere words, no matter from the heart.


import numpy as np
import random
import sys
import tensorflow as tf
import pandas as pd

from tensorflow.keras.callbacks import LambdaCallback
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM
from tensorflow.keras.optimizers import RMSprop

In [3]:
url = 'https://www.gutenberg.org/files/100/100-0.txt'
filepath = tf.keras.utils.get_file(fname='shakespeare.txt', origin=url)

5783552/5777367 [==============================] - 1s 0us/step


In [4]:
full_text = open(filepath, 'r', encoding='utf-8-sig').read()

In [7]:
sonnets = full_text[2771:101122]

In [10]:
sonnets[0:9898]

'THE SONNETS\n\n                    1\n\nFrom fairest creatures we desire increase,\nThat thereby beauty’s rose might never die,\nBut as the riper should by time decease,\nHis tender heir might bear his memory:\nBut thou contracted to thine own bright eyes,\nFeed’st thy light’s flame with self-substantial fuel,\nMaking a famine where abundance lies,\nThy self thy foe, to thy sweet self too cruel:\nThou that art now the world’s fresh ornament,\nAnd only herald to the gaudy spring,\nWithin thine own bud buriest thy content,\nAnd, tender churl, mak’st waste in niggarding:\n  Pity the world, or else this glutton be,\n  To eat the world’s due, by the grave and thee.\n\n\n                    2\n\nWhen forty winters shall besiege thy brow,\nAnd dig deep trenches in thy beauty’s field,\nThy youth’s proud livery so gazed on now,\nWill be a tattered weed of small worth held:\nThen being asked, where all thy beauty lies,\nWhere all the treasure of thy lusty days;\nTo say, within thine own deep su

In [34]:
def preprocess(text):
    text = ' '.join(text.split())
    return text

def make_xy(text):
    prep  = preprocess(text)
    chars = sorted(set(text))
    ' '.join(chars)
    n_chars = len(text)
    n_vocab = len(chars)
    maxlen = 40
    steps = 3

    subsequences = [] # X 
    next_chars = [] # Y

    for i in range(0, len(text) - maxlen, steps):
        subsequences.append(text[i : i + maxlen])
        next_chars.append(text[i + maxlen])

    x = np.zeros((len(subsequences), maxlen, len(chars)), dtype=np.bool)
    y = np.zeros((len(subsequences), len(chars)), dtype=np.bool)

    for i, subsequence in enumerate(subsequences):
        for t, char in enumerate(subsequence):
            x[i, t, char_to_index[char]] = 1

        y[i, char_to_index[next_chars[i]]] = 1
        
    return x, y


In [13]:
sonnets = preprocess(sonnets)

In [14]:
chars = sorted(set(sonnets))
' '.join(chars)

'  ! ( ) , - . 0 1 2 3 4 5 6 7 8 9 : ; ? A B C D E F G H I J K L M N O P R S T U V W Y a b c d e f g h i j k l m n o p q r s t u v w x y z ‘ ’'

In [15]:
char_to_index = {c:i for i, c in enumerate(chars)}
index_to_char = {i:c for i, c in enumerate(chars)}

In [16]:
n_chars = len(sonnets)
n_vocab = len(chars)
print ('Total Characters:', n_chars)
print ('Unique Characters:', n_vocab)

Total Characters: 94192
Unique Characters: 71


In [17]:
maxlen = 40
steps = 3

subsequences = [] # X 
next_chars = [] # Y

for i in range(0, len(sonnets) - maxlen, steps):
    subsequences.append(sonnets[i : i + maxlen])
    next_chars.append(sonnets[i + maxlen])

print ('Number of subsequences:', len(subsequences))

Number of subsequences: 31384


In [18]:
subsequences[0]

'THE SONNETS 1 From fairest creatures we '

In [24]:
# Specify x & y.
x = np.zeros((len(subsequences), maxlen, len(chars)), dtype=np.bool)
y = np.zeros((len(subsequences), len(chars)), dtype=np.bool)

for i, subsequence in enumerate(subsequences):
    for t, char in enumerate(subsequence):
        x[i, t, char_to_index[char]] = 1
        
    y[i, char_to_index[next_chars[i]]] = 1

In [20]:
x.shape

(31384, 40, 71)

In [25]:
model = Sequential()
model.add(LSTM(128, input_shape=(maxlen, len(chars))))
model.add(Dense(len(chars), activation='softmax'))

optimizer = RMSprop()
model.compile(loss='categorical_crossentropy', optimizer=optimizer)

In [26]:
def sample(preds, temperature=1.0):
    """
    Helper function to sample an index from a probability array.
    """
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

In [38]:
def on_epoch_end(epoch, _):
    # Function invoked at end of each epoch. Prints generated text.
    print()
    print('----- Generating text after Epoch: %d' % epoch)

    start_index = random.randint(0, len(sonnets) - maxlen - 1)
    for diversity in [0.2, 0.5, 1.0, 1.2]:
        print('----- diversity:', diversity)

        generated = ''
        subsequence = sonnets[start_index: start_index + maxlen]
        generated += subsequence
        print('----- Generating with seed: "' + subsequence + '"')
        sys.stdout.write(generated)

        for i in range(400):
            x_pred = np.zeros((1, maxlen, len(chars)))
            for t, char in enumerate(subsequence):
                x_pred[0, t, char_to_index[char]] = 1.

            preds = model.predict(x_pred, verbose=0)[0]
            next_index = sample(preds, diversity)
            next_char = index_to_char[next_index]

            subsequence = subsequence[1:] + next_char

            sys.stdout.write(next_char)
            sys.stdout.flush()
        print()

print_callback = LambdaCallback(on_epoch_end=on_epoch_end)

In [35]:
X, Y = make_xy(full_text)

MemoryError: 

In [39]:
model.fit(x, y,
          batch_size=128,
          epochs=5,
          callbacks=[print_callback])

Epoch 1/5
31360/31384 [============================>.] - ETA: 0s - loss: 2.2769
----- Generating text after Epoch: 0
----- diversity: 0.2
----- Generating with seed: "et-seasoned showers are to the ground; A"
et-seasoned showers are to the ground; And the thes the that the sores the the thin the that thou the here the the sing the ther the ther that the sorere the the thes the ther the ther the that the the that the sore the sing the sorere the heres the ther sore the the serer the thou ind the sore the sorthes the sthe here the the the the sore the the ther ther the the serer thou the then the here the the wore the that the wire the thet th
----- diversity: 0.5
----- Generating with seed: "et-seasoned showers are to the ground; A"
et-seasoned showers are to the ground; And me thee thee and thou porert wind shath routh al the lore the sheven thou hor sereres fore I lous th than th thal seweren tirhe this yris the serthen for thare forer dof tous whar the sares ard bith tho ghath the se

# Resources and Stretch Goals

## Stretch goals:
- Refine the training and generation of text to be able to ask for different genres/styles of Shakespearean text (e.g. plays versus sonnets)
- Train a classification model that takes text and returns which work of Shakespeare it is most likely to be from
- Make it more performant! Many possible routes here - lean on Keras, optimize the code, and/or use more resources (AWS, etc.)
- Revisit the news example from class, and improve it - use categories or tags to refine the model/generation, or train a news classifier
- Run on bigger, better data

## Resources:
- [The Unreasonable Effectiveness of Recurrent Neural Networks](https://karpathy.github.io/2015/05/21/rnn-effectiveness/) - a seminal writeup demonstrating a simple but effective character-level NLP RNN
- [Simple NumPy implementation of RNN](https://github.com/JY-Yoon/RNN-Implementation-using-NumPy/blob/master/RNN%20Implementation%20using%20NumPy.ipynb) - Python 3 version of the code from "Unreasonable Effectiveness"
- [TensorFlow RNN Tutorial](https://github.com/tensorflow/models/tree/master/tutorials/rnn) - code for training a RNN on the Penn Tree Bank language dataset
- [4 part tutorial on RNN](http://www.wildml.com/2015/09/recurrent-neural-networks-tutorial-part-1-introduction-to-rnns/) - relates RNN to the vanishing gradient problem, and provides example implementation
- [RNN training tips and tricks](https://github.com/karpathy/char-rnn#tips-and-tricks) - some rules of thumb for parameterizing and training your RNN